In [4]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from deap import base, creator, tools, algorithms
import random


In [5]:
# Load dataset
digits = load_digits()
X = digits.data
y = digits.target

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [6]:
def evaluate(individual):
    # Decode individual to hyperparameters
    hidden_layer_sizes = tuple([int(ind) for ind in individual[:2]])  # Use the first two genes for hidden layers
    activation = ["identity", "logistic", "tanh", "relu"][individual[2]]  # Use the third gene for activation function
    solver = ["lbfgs", "sgd", "adam"][individual[3]]  # Use the fourth gene for solver
    alpha = individual[4]  # Use the fifth gene for alpha

    # Initialize MLPClassifier with these hyperparameters
    mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes,
                        activation=activation,
                        solver=solver,
                        alpha=alpha,
                        max_iter=2000, random_state=42)
    
    # Train the model
    mlp.fit(X_train, y_train)
    
    # Predict and evaluate accuracy
    predictions = mlp.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    
    return (accuracy,)


In [9]:
# Define the individual (hyperparameters to be optimized)
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Define the gene ranges for each hyperparameter
toolbox.register("attr_hidden_layer_size", random.randint, 10, 200)
toolbox.register("attr_activation", random.randint, 0, 3)
toolbox.register("attr_solver", random.randint, 0, 2)
toolbox.register("attr_alpha", random.uniform, 0.0001, 0.01)

# Define the individual and population
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_hidden_layer_size,
                  toolbox.attr_hidden_layer_size,
                  toolbox.attr_activation,
                  toolbox.attr_solver,
                  toolbox.attr_alpha), n=1)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Register the evaluation, mating, mutation, and selection functions
toolbox.register("mate", tools.cxTwoPoint)
# Separate mutation functions: use `mutUniformInt` for integer genes and `mutGaussian` for float genes.
toolbox.register("mutate_hidden_layers", tools.mutUniformInt, low=10, up=200, indpb=0.2)
toolbox.register("mutate_activation_solver", tools.mutUniformInt, low=0, up=[3, 2], indpb=0.2)
toolbox.register("mutate_alpha", tools.mutGaussian, mu=0.0001, sigma=0.01, indpb=0.2)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)  # Use shuffle mutation for the whole individual
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

def mutate_individual(individual):
    # Apply the integer mutations
    individual[0:2] = tools.mutUniformInt(individual[0:2], low=[10, 10], up=[200, 200], indpb=0.2)[0]
    individual[2:4] = tools.mutUniformInt(individual[2:4], low=[0, 0], up=[3, 2], indpb=0.2)[0]
    # Apply the float mutation
    individual[4] = tools.mutGaussian([individual[4]], mu=0.0001, sigma=0.01, indpb=0.2)[0][0]
    return individual,

toolbox.register("mutate", mutate_individual)



c:\Users\Alireza\anaconda3\envs\recommendersystem\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\Alireza\anaconda3\envs\recommendersystem\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [10]:
def main():
    random.seed(42)
    
    # Initialize population
    population = toolbox.population(n=20)
    
    # Apply the genetic algorithm
    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=10, verbose=True)
    
    # Get the best individual
    best_individual = tools.selBest(population, k=1)[0]
    print("Best individual is:", best_individual)
    print("Best accuracy is:", evaluate(best_individual)[0])

if __name__ == "__main__":
    main()

gen	nevals
0  	20    
1  	8     
2  	13    
3  	16    
4  	10    
5  	10    
6  	14    
7  	12    
8  	15    
9  	10    
10 	14    
Best individual is: [173, 38, 2, 2, 0.002822790251854281]
Best accuracy is: 0.9888888888888889
